In [1]:
import numpy as np
import struct
import math
import os
from scipy.io import FortranFile
from config import *
import const

from functions import move_to_sim_dir, get_info


In [2]:
sim_round = 9
sim_name = 'fiducial'
dump = 54

move_to_sim_dir(sim_round, sim_name)
info = get_info(dump)
os.chdir('output_%05d' % dump)


Moving to directory '/home/za9132/scratch/romain/round9/fiducial'.


In [3]:
def hilbert3d(x, y, z, bitlen):
    
    state_diagram = [ 1, 2, 3, 2, 4, 5, 3, 5,
                      0, 1, 3, 2, 7, 6, 4, 5,
                      2, 6, 0, 7, 8, 8, 0, 7,
                      0, 7, 1, 6, 3, 4, 2, 5,
                      0, 9, 10, 9, 1, 1, 11, 11,
                      0, 3, 7, 4, 1, 2, 6, 5,
                      6, 0, 6, 11, 9, 0, 9, 8,
                      2, 3, 1, 0, 5, 4, 6, 7,
                      11,11, 0, 7, 5, 9, 0, 7,
                      4, 3, 5, 2, 7, 0, 6, 1,
                      4, 4, 8, 8, 0, 6,10, 6,
                      6, 5, 1, 2, 7, 4, 0, 3,
                      5, 7, 5, 3, 1, 1,11,11,
                      4, 7, 3, 0, 5, 6, 2, 1,
                      6, 1, 6, 10, 9, 4, 9, 10,
                      6, 7, 5, 4, 1, 0, 2, 3,
                      10, 3, 1, 1, 10, 3, 5, 9,
                      2, 5, 3, 4, 1, 6, 0, 7,
                      4, 4, 8, 8, 2, 7, 2, 3,
                      2, 1, 5, 6, 3, 0, 4, 7,
                      7, 2,11, 2, 7, 5, 8, 5,
                      4, 5, 7, 6, 3, 2, 0, 1,
                      10, 3, 2, 6, 10, 3, 4, 4,
                      6, 1, 7, 0, 5, 2, 4, 3]

    state_diagram = np.array(state_diagram)
    state_diagram = state_diagram.reshape((8, 2, 12), order='F')
    order = np.zeros_like(x)
    bit_mask = np.zeros(3*bitlen, dtype=bool)
    
    for i in range(len(x)):
        
        j = np.arange(bitlen)
        bit_mask[2::3] = x[i] & (1 << np.arange(bitlen))
        bit_mask[1::3] = y[i] & (1 << np.arange(bitlen))
        bit_mask[0::3] = z[i] & (1 << np.arange(bitlen))

        state = 0
        for j in range(bitlen-1, -1, -1):
            sdigit = np.sum(bit_mask[3*j:3*(j+1)] * 2**np.arange(3))
            bit_mask[3*j:3*(j+1)] = state_diagram[sdigit, 1, state] & (1 << np.arange(3))
            state = state_diagram[sdigit, 0, state]
            
        order[i] = np.sum(bit_mask * 2**np.arange(3*bitlen))

    return order


In [284]:
radius = 10*const.kpc
center = np.array([0, 0, 0])

ndim = 3
boxlen = info.aexp*100*const.Mpc/const.h0
radius_code = radius/info.length_unit
center_code = center/info.length_unit + np.array([0.5, 0.5, 0.5])

level_min = np.max([1, math.floor(np.log2(1/radius_code/2))])
level_max = info.amr_level_sim_max
bitlen = level_min-1

dkey = 2**(ndim*(level_max-level_min))
ibound = [0, 0, 0, 0, 0, 0]

if (bit_len > 0):
    ibound[0:3] = np.array((center_code-radius_code)*2**bitlen, dtype=int)
    ibound[3:6] = np.array((center_code+radius_code)*2**bitlen, dtype=int)
    ndom = 8
    idom = [ibound[0], ibound[3]]*4
    jdom = ([ibound[1]]*2 + [ibound[4]]*2)*2
    kdom = [ibound[2]]*4 + [ibound[5]]*4
    order_min = hilbert3d(idom, jdom, kdom, bitlen)
else:
    ndom = 1
    order_min = np.array([0.])
    
bounding_min = order_min*dkey
bounding_max = (order_min+1)*dkey

cpu_min = np.zeros(ndom, dtype=int)
cpu_max = np.zeros(ndom, dtype=int)
for idx_cpu in range(0, info.ncpu):
    for idom in range(0, ndom):
        if( (info.bound_key[icpu] <= bounding_min[idom]) and (info.bound_key[icpu+1] > bounding_min[idom]) ):
            cpu_min[idom] = icpu+1
        if( (info.bound_key[icpu] < bounding_max[idom]) and (info.bound_key[icpu+1] >= bounding_max[idom]) ):
            cpu_max[idom] = icpu+1


ncpu_read = 0
cpu_read = np.zeros(ncpu, dtype=bool)
cpu_list = []
for idom in range(0,ndom):
    for icpu in range(cpu_min[idom]-1,cpu_max[idom]):
        if ( not cpu_read[icpu] ):
            cpu_list.append(icpu+1)
            ncpu_read = ncpu_read+1
            cpu_read[icpu] = True


array([ 1414745, 15362470,  7556404,  9220811,  3828134, 12949081,
        4560473, 12216742])

In [285]:
def get_cpu_list(center, radius):
    
    for i in range(0, info.nlevelmax):
        dx = 1/2**ilevel
        if (dx < 2*radius/info.boxlen):
            break

    levelmin = np.max([ilevel,1])
    bit_length = levelmin-1
    nmax = 2**bit_length
    ndim = info.ndim
    ncpu = info.ncpu
    nlevelmax = info.nlevelmax
    dkey = 2**(ndim*(nlevelmax+1-bit_length))
    ibound = [0, 0, 0, 0, 0, 0]
    if(bit_length > 0):
        ibound[0:3] = (center-radius)*nmax/info.boxlen
        ibound[3:6] = (center+radius)*nmax/info.boxlen
        ibound[0:3] = np.array(ibound[0:3]).astype(int)
        ibound[3:6] = np.array(ibound[3:6]).astype(int)
        ndom = 8
        idom = [ibound[0], ibound[3], ibound[0], ibound[3], ibound[0], ibound[3], ibound[0], ibound[3]]
        jdom = [ibound[1], ibound[1], ibound[4], ibound[4], ibound[1], ibound[1], ibound[4], ibound[4]]
        kdom = [ibound[2], ibound[2], ibound[2], ibound[2], ibound[5], ibound[5], ibound[5], ibound[5]]
        order_min = hilbert3d(idom,jdom,kdom,bit_length)
    else:
        ndom = 1
        order_min = np.array([0.])
        
    bounding_min = order_min*dkey
    bounding_max = (order_min+1)*dkey

    cpu_min = np.zeros(ndom, dtype=int)
    cpu_max = np.zeros(ndom, dtype=int)
    for icpu in range(0,ncpu):
        for idom in range(0,ndom):
            if( (info.bound_key[icpu] <= bounding_min[idom]) and (info.bound_key[icpu+1] > bounding_min[idom]) ):
                cpu_min[idom] = icpu+1
            if( (info.bound_key[icpu] < bounding_max[idom]) and (info.bound_key[icpu+1] >= bounding_max[idom]) ):
                cpu_max[idom] = icpu+1


    ncpu_read = 0
    cpu_read = np.zeros(ncpu, dtype=bool)
    cpu_list = []
    for idom in range(0,ndom):
        for icpu in range(cpu_min[idom]-1,cpu_max[idom]):
            if ( not cpu_read[icpu] ):
                cpu_list.append(icpu+1)
                ncpu_read = ncpu_read+1
                cpu_read[icpu] = True

    return cpu_list


In [59]:
get_cpu_list(0, 10)


AttributeError: 'types.SimpleNamespace' object has no attribute 'nlevelmax'

In [26]:
id_star = np.array([])
    tau_starbirth = np.array([])
    for i in range(1, info.ncpu+1):
        partfile = os.path.join("output_%.5d" % dump, "part_%.5d.out%.5d" % (dump, i))
        with FortranFile(partfile, 'r') as f:
            _, _, _ = f.read_ints('i'), f.read_ints('i'), f.read_ints('i')
            _, _, _, _, _ = f.read_reals('f8'), f.read_reals('f4'), f.read_reals('f8'), f.read_reals('f8'), f.read_reals('f4')
            _, _, _ = f.read_reals('f8'), f.read_reals('f8'), f.read_reals('f8') # position
            _, _, _ = f.read_reals('f8'), f.read_reals('f8'), f.read_reals('f8') # velocity
            _ = f.read_reals('f8') # mass
            id_part = f.read_ints('i') # id
            _ = f.read_ints('i') # level
            type_part = f.read_ints('b') # family
            _ = f.read_ints('b') # tag
            tau_partbirth = f.read_reals('f8') # birthtime
            _ = f.read_reals('f8') # metallicity
            id_star = np.concatenate((id_star, id_part[type_part==STAR]))
            tau_starbirth = np.concatenate((tau_starbirth, tau_partbirth[type_part==STAR]))
    integrand = lambda a: (info.Omega_m0 * a**(-1) + info.Omega_k0 + info.Omega_L0 * a**2)**(-1/2)
    age_universe = quad(integrand, 0, 1)[0] / info.H0
    time_starbirth = tau_starbirth / info.H0 + age_universe
    return id_star, time_starbirth


In [31]:
read_partfile(54)


NameError: name 'quad' is not defined